# Day 1: Logistic Regression
We will apply logistic regression to a binary classification problem. We will try out a few datasets, some randomly generated (i.e. artificial) and some real-life data.

Most packages will also have datasets alongwith. For instance the statsmodels package has a whole bunch of datasets as in http://statsmodels.sourceforge.net/devel/datasets/index.html

The next example is based on the following site that uses Fair's data on extramarital affairs, see
http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/discrete_choice_example.html

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [2]:
from __future__ import print_function

The __future__ module is a real Python module. You will often see constructs like from __future__ import division. What do you think this line does? 

In [3]:
print(sm.datasets.fair.SOURCE)


Fair, Ray. 1978. "A Theory of Extramarital Affairs," `Journal of Political
    Economy`, February, 45-61.

The data is available at http://fairmodel.econ.yale.edu/rayfair/pdf/2011b.htm



In [5]:
print( sm.datasets.fair.NOTE)

::

    Number of observations: 6366
    Number of variables: 9
    Variable name definitions:

        rate_marriage   : How rate marriage, 1 = very poor, 2 = poor, 3 = fair,
                        4 = good, 5 = very good
        age             : Age
        yrs_married     : No. years married. Interval approximations. See
                        original paper for detailed explanation.
        children        : No. children
        religious       : How relgious, 1 = not, 2 = mildly, 3 = fairly,
                        4 = strongly
        educ            : Level of education, 9 = grade school, 12 = high
                        school, 14 = some college, 16 = college graduate,
                        17 = some graduate school, 20 = advanced degree
        occupation      : 1 = student, 2 = farming, agriculture; semi-skilled,
                        or unskilled worker; 3 = white-colloar; 4 = teacher
                        counselor social worker, nurse; artist, writers;
          

Load the dataset using Pandas. Note that this is a Pandas data structure, so we have to use Pandas' methods in order to access the data (say, print the first line of the data table). 

In [15]:
dta = sm.datasets.fair.load_pandas().data 
# Try out print(dta) 
print(dta.head(1))

   rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   

   occupation_husb   affairs  
0              5.0  0.111111  


__Sidenote__: Here, print dta.head(1) will not work. This is because we imported print as a function, see the line from __future__ import print_function, so print a will not work any more, since this is using print as a statement.

In [20]:
#print(dta['affairs']) 

Note that the affairs column shows the total amount of time spent in affairs. Let us try to classify whether a person will cheat or not. Hence, let us replace all entries where dta['affairs'] > 0 by 1 (rather by 1.0). We can get this by the Boolean expression (dta['affairs']> 0). 

We will make this into a new column, and call the new column "affair"

In [22]:
# (dta['affairs']> 0)
dta['affair'] = (dta['affairs'] > 0).astype(float)

   rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   
1            3.0  27.0         13.0       3.0        1.0  14.0         3.0   

   occupation_husb   affairs  affair  
0              5.0  0.111111     1.0  
1              4.0  3.230769     1.0  


In [28]:
print(dta.head(2))

   rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   
1            3.0  27.0         13.0       3.0        1.0  14.0         3.0   

   occupation_husb   affairs  affair  
0              5.0  0.111111     1.0  
1              4.0  3.230769     1.0  


People who use R will be familiar with the next few steps. We want to "describe" the dataset, i.e. get the most relevant statistics to understand this data.

In [29]:
print(dta.describe())

       rate_marriage          age  yrs_married     children    religious  \
count    6366.000000  6366.000000  6366.000000  6366.000000  6366.000000   
mean        4.109645    29.082862     9.009425     1.396874     2.426170   
std         0.961430     6.847882     7.280120     1.433471     0.878369   
min         1.000000    17.500000     0.500000     0.000000     1.000000   
25%         4.000000    22.000000     2.500000     0.000000     2.000000   
50%         4.000000    27.000000     6.000000     1.000000     2.000000   
75%         5.000000    32.000000    16.500000     2.000000     3.000000   
max         5.000000    42.000000    23.000000     5.500000     4.000000   

              educ   occupation  occupation_husb      affairs       affair  
count  6366.000000  6366.000000      6366.000000  6366.000000  6366.000000  
mean     14.209865     3.424128         3.850141     0.705374     0.322495  
std       2.178003     0.942399         1.346435     2.203374     0.467468  
min    